# Intro
Initial exploration of the dataset

# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_dark"
from src import data_wrangler

%load_ext autoreload
%autoreload 2

# Dataset

In [8]:
sales = data_wrangler.get_sales_data()

In [9]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
order_id,,,,,,,,,,
0,2013-01-02,0,59,22154,999.00,1.0,Scene 2012 (BD),37,"Yaroslavl shopping center ""Altair""",Movies - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,"Moscow TRC ""Atrium""",Music - Vinyl
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Moscow TRC ""Atrium""",Music - CD production firm


## Orders per date
Each row of the dataset is an order

In [10]:
px.line(
    sales.date.value_counts().sort_index().to_frame("orders"),
    labels={"value": "orders", "index": "day"},
)

### Date block num
The dataset contains a column called `date_block_num`, which means:

*a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33*

In [20]:
sales.groupby(sales.date.dt.to_period("M")).date_block_num.nunique().loc[
    lambda x: x != 1
]

Series([], Freq: M, Name: date_block_num, dtype: int64)

## Shops
Each one of the shops have at least one sale. 
There are 60 shops, with IDs ranging from 0 to 59


In [27]:
sales_per_shop = sales.shop_id.value_counts().sort_index()
px.bar(sales_per_shop)

In [29]:
set(range(0, 60)).issubset(sales_per_shop.index.tolist())

True

### Shops timelines
Shops born and die at different times

In [34]:
px.timeline(
    sales.groupby("shop_id").date.agg(["min", "max"]).reset_index().sort_values("min"),
    x_start="min",
    x_end="max",
    y="shop_id",
).update_yaxes(type='category')

## Percentage of orders per shop

In [37]:
px.bar(
    sales.shop_id.value_counts(normalize=True).sort_values(ascending=False),labels={
        'value':'%'
    }
).update_xaxes(type="category")

## Item ID

Not every item is sold, and some items are only sold once

In [51]:
sales.item_id.agg(['min','max'])

min        0
max    22169
Name: item_id, dtype: int64

In [50]:
set(range(0,22170)).issubset(sales.item_id.unique().tolist())

False

In [45]:
px.line(sales.item_id.value_counts()).update_xaxes(type='category')

## Item Price

In [60]:
sales.item_price.describe()

count    2.935849e+06
mean     8.908532e+02
std      1.729800e+03
min     -1.000000e+00
25%      2.490000e+02
50%      3.990000e+02
75%      9.990000e+02
max      3.079800e+05
Name: item_price, dtype: float64

## Item Count per Day

In [43]:
sales.item_cnt_day.describe

count    2.935849e+06
mean     1.242641e+00
std      2.618834e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.169000e+03
Name: item_cnt_day, dtype: float64

In [44]:
sales.item_cnt_day.lt(1).value_counts()

False    2928493
True        7356
Name: item_cnt_day, dtype: int64

In [47]:
sales.item_cnt_day.gt(1).value_counts(normalize=True)

False    0.898114
True     0.101886
Name: item_cnt_day, dtype: float64

In [61]:
pd.cut(sales.item_cnt_day, np.arange(-1, 10, 1)).value_counts(
    normalize=True, dropna=False, sort=False
)

(-1.0, 0.0]    0.000000
(0.0, 1.0]     0.895609
(1.0, 2.0]     0.066148
(2.0, 3.0]     0.016128
(3.0, 4.0]     0.006705
(4.0, 5.0]     0.003568
(5.0, 6.0]     0.002159
(6.0, 7.0]     0.001382
(7.0, 8.0]     0.000989
(8.0, 9.0]     0.000742
NaN            0.006571
Name: item_cnt_day, dtype: float64

## Item Category


In [93]:
sales.item_category_name.value_counts()

Кино - DVD                             564652
Игры PC - Стандартные издания          351591
Музыка - CD локального производства    339585
Игры - PS3                             208219
Кино - Blu-Ray                         192674
                                        ...  
Книги - Путеводители                        3
Книги - Открытки                            2
Аксессуары - PS2                            2
Игровые консоли - PS2                       1
Книги - Познавательная литература           1
Name: item_category_name, Length: 84, dtype: int64

In [98]:
sales.groupby(
    ["date_block_num", "item_category_name"]
).item_cnt_day.sum().reset_index().plot.hist(
    x="date_block_num", y="item_cnt_day", color="item_category_name"
)